In [41]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os 
import copy
import glob
import pickle

import numpy as np 

import astropy.units as u
from astropy.table import Table, Column, join, vstack

import healpy as hp

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def filter_healpix_mask(mask, catalog, ra='ra', dec='dec', verbose=True):
    """Filter a catalog through a Healpix mask.

    Parameters
    ----------
    mask : healpy mask data
        healpy mask data
    catalog : numpy array or astropy.table
        Catalog that includes the coordinate information
    ra : string
        Name of the column for R.A.
    dec : string
        Name of the column for Dec.
    verbose : boolen, optional
        Default: True

    Return
    ------
        Selected objects that are covered by the mask.
    """
    import healpy

    nside, hp_indices = healpy.get_nside(mask), np.where(mask)[0]

    phi, theta = np.radians(catalog[ra]), np.radians(90. - catalog[dec])

    hp_masked = healpy.ang2pix(nside, theta, phi, nest=True)

    select = np.in1d(hp_masked, hp_indices)

    if verbose:
        print("# %d/%d objects are selected by the mask" % (select.sum(), len(catalog)))

    return catalog[select]

In [3]:
sweep = '/Users/song/work/merian/sweep-000m025-010m020.fits'

cat = Table.read(sweep)

print(cat.colnames)

['RELEASE', 'BRICKID', 'BRICKNAME', 'OBJID', 'TYPE', 'RA', 'DEC', 'RA_IVAR', 'DEC_IVAR', 'DCHISQ', 'EBV', 'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FLUX_W1', 'FLUX_W2', 'FLUX_W3', 'FLUX_W4', 'FLUX_IVAR_G', 'FLUX_IVAR_R', 'FLUX_IVAR_Z', 'FLUX_IVAR_W1', 'FLUX_IVAR_W2', 'FLUX_IVAR_W3', 'FLUX_IVAR_W4', 'MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z', 'MW_TRANSMISSION_W1', 'MW_TRANSMISSION_W2', 'MW_TRANSMISSION_W3', 'MW_TRANSMISSION_W4', 'NOBS_G', 'NOBS_R', 'NOBS_Z', 'NOBS_W1', 'NOBS_W2', 'NOBS_W3', 'NOBS_W4', 'RCHISQ_G', 'RCHISQ_R', 'RCHISQ_Z', 'RCHISQ_W1', 'RCHISQ_W2', 'RCHISQ_W3', 'RCHISQ_W4', 'FRACFLUX_G', 'FRACFLUX_R', 'FRACFLUX_Z', 'FRACFLUX_W1', 'FRACFLUX_W2', 'FRACFLUX_W3', 'FRACFLUX_W4', 'FRACMASKED_G', 'FRACMASKED_R', 'FRACMASKED_Z', 'FRACIN_G', 'FRACIN_R', 'FRACIN_Z', 'ANYMASK_G', 'ANYMASK_R', 'ANYMASK_Z', 'ALLMASK_G', 'ALLMASK_R', 'ALLMASK_Z', 'WISEMASK_W1', 'WISEMASK_W2', 'PSFSIZE_G', 'PSFSIZE_R', 'PSFSIZE_Z', 'PSFDEPTH_G', 'PSFDEPTH_R', 'PSFDEPTH_Z', 'GALDEPTH_G', 'GALDEPTH

In [7]:
s20a_mask = '/Users/song/work/merian/s20a_fdfc_hp_contarea_izy-gt-5.fits'

s20a_fdfc = hp.read_map(s20a_mask, nest=True, dtype=np.bool)

<ipython-input-7-f3255d9e274c>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  s20a_fdfc = hp.read_map(s20a_mask, nest=True, dtype=np.bool)
/usr/local/lib/python3.8/site-packages/healpy/fitsfunc.py:391: UserWarning: NSIDE = 1024
  warnings.warn("NSIDE = {0:d}".format(nside))
/usr/local/lib/python3.8/site-packages/healpy/fitsfunc.py:400: UserWarning: ORDERING = NESTED in fits file
  warnings.warn("ORDERING = {0:s} in fits file".format(ordering))
/usr/local/lib/python3.8/site-packages/healpy/fitsfunc.py:428: UserWarning: INDXSCHM = IMPLICIT
  warnings.warn("INDXSCHM = {0:s}".format(schm))


In [9]:
cat_fdfc = filter_healpix_mask(s20a_fdfc, cat, ra='RA', dec='DEC', verbose=True)

# 0/1761713 objects are selected by the mask


`/tigress/MERIAN/ancillary/decals/sweep/9.0`

In [11]:
len(cat_fdfc)

0

In [12]:
cat_dir = '/Users/song/astro6/hsc/s20a/decals'

In [29]:
dr9_list = glob.glob(cat_dir + '/*.fits')

In [13]:
cat_test = Table.read(os.path.join(cat_dir, 'sweep-000m005-010p000_s20a.fits'))

In [35]:
cols = ['OBJID', 'TYPE', 'RA', 'DEC', 
        'FLUX_G', 'FLUX_R', 'FLUX_Z', 
        'FLUX_IVAR_G', 'FLUX_IVAR_R', 'FLUX_IVAR_Z', 
        'MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z', 
        'NOBS_G', 'NOBS_R', 'NOBS_Z', 
        'RCHISQ_G', 'RCHISQ_R', 'RCHISQ_Z', 
        'FRACFLUX_G', 'FRACFLUX_R', 'FRACFLUX_Z', 
        'FRACMASKED_G', 'FRACMASKED_R', 'FRACMASKED_Z', 
        'FRACIN_G', 'FRACIN_R', 'FRACIN_Z', 
        'ANYMASK_G', 'ANYMASK_R', 'ANYMASK_Z', 
        'ALLMASK_G', 'ALLMASK_R', 'ALLMASK_Z', 
        'PSFSIZE_G', 'PSFSIZE_R', 'PSFSIZE_Z', 
        'GALDEPTH_G', 'GALDEPTH_R', 'GALDEPTH_Z', 
        'SHAPE_R', 'SHAPE_R_IVAR', 'SHAPE_E1', 'SHAPE_E1_IVAR', 'SHAPE_E2', 'SHAPE_E2_IVAR', 
        'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 
        'FIBERTOTFLUX_G', 'FIBERTOTFLUX_R', 'FIBERTOTFLUX_Z', 
        'SERSIC', 'SERSIC_IVAR']

In [38]:
def reduce_cat(cat):
    gal_mask = cat['TYPE'] != 'PSF'
    
    flux_mask = (
        (cat['FLUX_G'] >= 0) & (cat['FLUX_R'] >= 0) & (cat['FLUX_Z'] >= 0) &
        (np.isfinite(cat['FLUX_G'])) & (np.isfinite(cat['FLUX_R'])) & 
        (np.isfinite(cat['FLUX_Z']))
    )

    r_mag = 22.5 - 2.5 * np.log10(cat['FLUX_R'])
    r_mask = r_mag <= 24.0

    print((gal_mask & flux_mask & r_mask).sum(), len(cat))
    
    return cat[(gal_mask & flux_mask & r_mask)][cols]
    

In [40]:
dr9_reduced = [reduce_cat(Table.read(c)) for c in dr9_list]

<ipython-input-38-e2e6b8644b2d>:10: RuntimeWarning: divide by zero encountered in log10
  r_mag = 22.5 - 2.5 * np.log10(cat['FLUX_R'])
<ipython-input-38-e2e6b8644b2d>:10: RuntimeWarning: invalid value encountered in log10
  r_mag = 22.5 - 2.5 * np.log10(cat['FLUX_R'])


75938 208500
286463 810045
144816 399736
28735 80477
62315 171264
2293900 6132037
529396 1499798
888949 2524949
77556 178761
252388 554739
602641 1273911
1890701 3923273
601179 1193298
1480081 2922713
472699 943466
998999 2013480
457127 929089
504508 1021119
601462 1183705
1273871 2476770
600106 1208492
1126778 2266762
567162 1200968
456118 959163
592141 1263287
416018 893980
611134 1311837
479706 1015639
319770 737654
237226 527492
249029 744627
1874528 4852079
132591 295043
273509 813596
1249872 3381428
39232 87210
273806 817890
971250 2727344


In [42]:
dr9_hsc = vstack(dr9_reduced)

In [43]:
len(dr9_hsc)

23993700

In [44]:
dr9_hsc.write(os.path.join(cat_dir, 'decals_dr9_hsc_s20a_gal_r24.fits'), overwrite=True)